# Importing Librarys

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix,classification_report

# Loading IMDB Dataset

In [2]:
ds = tfds.load('imdb_reviews')
ds

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete6I2BNN/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete6I2BNN/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete6I2BNN/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


{'test': <PrefetchDataset shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>,
 'train': <PrefetchDataset shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>,
 'unsupervised': <PrefetchDataset shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>}

**Converting to Dataframe**

In [3]:
train = tfds.as_dataframe(ds['train'])
test = tfds.as_dataframe(ds['test'])
train.head(5)

,label,text
0,0,"b""This was an absolutely terrible movie. Don't..."
1,0,b'I have been known to fall asleep during film...
2,0,b'Mann photographs the Alberta Rocky Mountains...
3,1,b'This is the kind of film for a snowy Sunday ...
4,1,"b'As others have mentioned, all the women that..."


# Converting bytes to string

In [4]:
# byte to string
def byte_to_string(byte): return byte.decode("utf-8")

In [5]:
train['text'] =  train['text'].apply(byte_to_string) 
test['text']  = test['text'].apply(byte_to_string) 

In [6]:
train.head(5)

,label,text
0,0,This was an absolutely terrible movie. Don't b...
1,0,"I have been known to fall asleep during films,..."
2,0,Mann photographs the Alberta Rocky Mountains i...
3,1,This is the kind of film for a snowy Sunday af...
4,1,"As others have mentioned, all the women that g..."


# Class Distribution

In [7]:
train['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

# Loading Universal Encoder Layer

In [8]:
embedding = "https://tfhub.dev/google/universal-sentence-encoder/4"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

# Model Architecture

In [9]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(128, activation='elu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 512)               256797824 
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               65664     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 256,863,617
Trainable params: 256,863,617
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])

# Model Training

In [11]:
history = model.fit(train['text'], train['label'], validation_split = 0.2 , shuffle= True, epochs=4,batch_size=1024)

Epoch 1/4
20/20 [==============================] - 40s 2s/step - loss: 0.5493 - accuracy: 0.7503 - val_loss: 0.3622 - val_accuracy: 0.8484
Epoch 2/4
20/20 [==============================] - 32s 2s/step - loss: 0.2735 - accuracy: 0.8891 - val_loss: 0.2777 - val_accuracy: 0.8812
Epoch 3/4
20/20 [==============================] - 32s 2s/step - loss: 0.1146 - accuracy: 0.9610 - val_loss: 0.3177 - val_accuracy: 0.8838
Epoch 4/4
20/20 [==============================] - 31s 2s/step - loss: 0.0297 - accuracy: 0.9929 - val_loss: 0.3199 - val_accuracy: 0.8928


# Test Set Prediction

**prediction**

In [16]:
prediction = model.predict(test['text'])>0.5

**Confusion Matrix**

In [17]:
confusion_matrix(np.array(test['label']),prediction)

array([[11436,  1064],
       [ 1798, 10702]])

**Classification Report**

In [18]:
print(classification_report(np.array(test['label']),prediction,labels=[0,1]))

              precision    recall  f1-score   support

           0       0.86      0.91      0.89     12500
           1       0.91      0.86      0.88     12500

    accuracy                           0.89     25000
   macro avg       0.89      0.89      0.89     25000
weighted avg       0.89      0.89      0.89     25000

